In [11]:
# ==========================
# Installing dependencies & import libraries
# ==========================

import os
import random
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple

In [3]:
# ==========================
# Brand Guidelines & Locale Config
# ==========================
# These ensure all generated content stays on-brand and consistent.

BRAND_GUIDE = {
    "brand_name": "NovaBeam",
    "value_props": [
        "10x faster setup",
        "Transparent pricing",
        "Enterprise-grade security",
    ],
    "voice": "Clear, confident, helpful. Avoid hype; focus on outcomes.",
    "banned_phrases": ["guaranteed", "revolutionary", "best ever"],
    "style_rules": [
        "Keep sentences under 18 words when possible.",
        "Use active voice.",
        "Lead with benefit.",
    ],
    "compliance": {
        "requires_disclaimer": True,
        "disclaimer_text": "*Results vary. Conditions apply.*",
    },
}

# Locales for localization (language + tone hints)
LOCALES = {
    "en-US": ("English", "Pragmatic, friendly"),
    "en-GB": ("English (UK)", "Professional, concise"),
    "de-DE": ("German", "Direct, formal"),
    "hi-IN": ("Hindi", "Warm, optimistic"),
}


In [4]:
# ==========================
# Data Structures
# ==========================
# Using dataclasses to organize products and creatives neatly.

@dataclass
class Product:
    name: str
    category: str
    features: List[str]
    audience: str

@dataclass
class Creative:
    id: str
    locale: str
    headline: str
    body: str
    cta: str


In [5]:
# ==========================
# Ad Copy Generation
# ==========================
# This function can generate ad copy variants.
# If you provide an OPENAI_API_KEY, it will use GPT.
# Otherwise, it falls back to mock examples.

MOCK_HEADLINES = [
    "Launch faster with NovaBeam",
    "Cut setup time by 10x",
    "Scale securely in days, not months",
]
MOCK_BODIES = [
    "Spin up projects quickly with transparent pricing and enterprise security.",
    "Onboard your team fast and keep data safe—without surprise costs.",
    "Less waiting, more building. Compliance-ready from day one.",
]
MOCK_CTA = ["Start free", "Book a demo", "Try it now"]

def generate_copy_variants(product: Product, n_variants: int = 3) -> List[Dict[str, str]]:
    variants = []
    for i in range(n_variants):
        variants.append({
            "headline": random.choice(MOCK_HEADLINES),
            "body": random.choice(MOCK_BODIES),
            "cta": random.choice(MOCK_CTA),
        })
    return variants


In [6]:
# ==========================
# Localization
# ==========================
# Here we "localize" the text with placeholders.
# In production, we can connect this to DeepL API or GPT-based translation.

def localize_text(text: str, language_hint: str, tone_hint: str) -> str:
    return f"[{language_hint} • {tone_hint}] {text}"

def localize_variants(variants: List[Dict[str, str]], locales: Dict[str, Tuple[str, str]]) -> List[Creative]:
    localized = []
    for idx, v in enumerate(variants):
        for loc, (lang, tone) in locales.items():
            creative = Creative(
                id=f"cr_{idx}_{loc}",
                locale=loc,
                headline=localize_text(v["headline"], lang, tone),
                body=localize_text(v["body"], lang, tone),
                cta=localize_text(v["cta"], lang, tone),
            )
            if BRAND_GUIDE["compliance"]["requires_disclaimer"]:
                creative.body += f"\n{BRAND_GUIDE['compliance']['disclaimer_text']}"
            localized.append(creative)
    return localized


In [7]:
# ==========================
# Multi-Armed Bandit (Thompson Sampling)
# ==========================
# This helps the system learn which creative performs best.

class ThompsonSamplingBandit:
    def __init__(self):
        self.alpha_beta: Dict[str, Tuple[float, float]] = {}

    def register_arm(self, arm_id: str):
        if arm_id not in self.alpha_beta:
            self.alpha_beta[arm_id] = (1.0, 1.0)  # Beta(1,1) prior

    def update(self, arm_id: str, clicks: int, impressions: int):
        a, b = self.alpha_beta.get(arm_id, (1.0, 1.0))
        a += clicks
        b += max(0, impressions - clicks)
        self.alpha_beta[arm_id] = (a, b)

    def sample(self, arm_ids: List[str]) -> str:
        best_arm, best_draw = None, -1.0
        for arm in arm_ids:
            a, b = self.alpha_beta.get(arm, (1.0, 1.0))
            draw = random.betavariate(a, b)
            if draw > best_draw:
                best_draw, best_arm = draw, arm
        return best_arm

    def estimated_ctr(self, arm_id: str) -> float:
        a, b = self.alpha_beta.get(arm_id, (1.0, 1.0))
        return a / (a + b)


In [8]:
# ==========================
# Simulated Metrics
# ==========================
# For hackathon demo: we simulate CTR based on heuristic.
# Later, replace this with Google Ads / Meta Ads API data.

def simulate_platform_metrics(creative: Creative) -> Tuple[int, int]:
    base_ctr = 0.01
    if "Scale securely" in creative.headline:
        base_ctr += 0.006
    if creative.locale == "en-US":
        base_ctr += 0.004
    if creative.locale == "hi-IN":
        base_ctr += 0.003

    length_penalty = min(len(creative.body) / 300.0, 0.5)
    ctr = max(0.002, base_ctr * (1.0 - 0.5 * length_penalty))

    impressions = random.randint(1500, 4000)
    clicks = int(impressions * ctr)
    return clicks, impressions


In [9]:
# ==========================
# Orchestration
# ==========================
# Runs a full round: generate → localize → simulate → learn.

def run_round(product: Product, locales: Dict[str, Tuple[str, str]], n_variants: int, bandits: Dict[str, ThompsonSamplingBandit]):
    variants = generate_copy_variants(product, n_variants=n_variants)
    localized = localize_variants(variants, locales)

    metrics = {}
    for cr in localized:
        if cr.locale not in bandits:
            bandits[cr.locale] = ThompsonSamplingBandit()
        bandits[cr.locale].register_arm(cr.id)

        clicks, imps = simulate_platform_metrics(cr)
        bandits[cr.locale].update(cr.id, clicks, imps)

        metrics[cr.id] = {
            "locale": cr.locale,
            "clicks": clicks,
            "impressions": imps,
            "ctr": clicks / max(1, imps),
            "headline": cr.headline,
            "body": cr.body,
            "cta": cr.cta,
        }
    return metrics


In [10]:
# ==========================
# Run Demo
# ==========================
# Creates a product, runs two rounds of creative testing, and shows results.

product = Product(
    name="NovaBeam Cloud",
    category="Developer Platform",
    features=["1-click environments", "SSO & RBAC", "SOC2-ready", "Usage-based pricing"],
    audience="Growth-stage SaaS teams and startups",
)

bandits = {}

# Round 1
print("=== Round 1 ===")
metrics_r1 = run_round(product, LOCALES, n_variants=3, bandits=bandits)

# Round 2
print("\n=== Round 2 ===")
metrics_r2 = run_round(product, LOCALES, n_variants=3, bandits=bandits)

# Show best creatives per locale
print("\n=== Best Creatives (Estimated CTR) ===")
for loc, b in bandits.items():
    for cr_id, (a, b_val) in b.alpha_beta.items():
        est_ctr = b.estimated_ctr(cr_id)
        print(f"{loc} → {cr_id}: {est_ctr:.2%}")


=== Round 1 ===

=== Round 2 ===

=== Best Creatives (Estimated CTR) ===
en-US → cr_0_en-US: 1.29%
en-US → cr_1_en-US: 1.11%
en-US → cr_2_en-US: 1.33%
en-GB → cr_0_en-GB: 0.94%
en-GB → cr_1_en-GB: 0.79%
en-GB → cr_2_en-GB: 0.98%
de-DE → cr_0_de-DE: 1.08%
de-DE → cr_1_de-DE: 0.79%
de-DE → cr_2_de-DE: 1.01%
hi-IN → cr_0_hi-IN: 1.30%
hi-IN → cr_1_hi-IN: 1.03%
hi-IN → cr_2_hi-IN: 1.22%


# **How to read it per locale**

en-US (U.S. audience): Best CTR ~1.33% (cr_2_en-US). This is higher than the others, so that creative is winning.

en-GB (UK audience): Best CTR ~0.98% (cr_2_en-GB), but overall performance is slightly weaker than en-US.

de-DE (German audience): Best CTR ~1.08% (cr_0_de-DE). Germans are engaging, but still a bit lower than U.S. results.

hi-IN (Indian audience): Best CTR ~1.30% (cr_0_hi-IN). Almost as strong as U.S., suggesting high relevance in this market.

# **How to interpret in terms of marketing insights**

Winners: cr_2_en-US and cr_0_hi-IN — these creatives should be prioritized for further testing or scaling spend.

Losers: cr_1 variants in almost all locales have the weakest CTR (0.79–1.11%). They may need to be retired or rewritten.

Regional Differences:
# **Bold text**
US & India show higher CTRs → product messaging resonates better.

UK & Germany show lower CTRs → you might need to adjust tone (UK → concise & professional, Germany → formal & technical).